In [1]:
import pandas as pd

In [2]:
# user_log = pd.read_csv('C:/Stats Consulting/user_log.csv', iterator=True, chunksize=10000)
user_log = pd.read_csv('/Users/joe/study/2401-statistical consulting/data/user_log_data.csv', iterator=True, chunksize=10000)
user_log = pd.concat(user_log, ignore_index=True)

In [3]:
user_log.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0


In [4]:
# user_info = pd.read_csv('C:/Stats Consulting/user_info.csv', iterator=True, chunksize=10000)
user_info = pd.read_csv('/Users/joe/study/2401-statistical consulting/data/user_info_data.csv', iterator=True, chunksize=10000)
user_info = pd.concat(user_info, ignore_index=True)
user_info.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


In [5]:
user_log.isnull().any()

user_id        False
item_id        False
cat_id         False
seller_id      False
brand_id        True
time_stamp     False
action_type    False
dtype: bool

In [6]:
user_info.isnull().any()

user_id      False
age_range     True
gender        True
dtype: bool

In [7]:
from sklearn.preprocessing import label_binarize

In [8]:
import numpy as np

## User profile

In [9]:
user_gender = user_info[['user_id','gender']]
user_gender = user_gender.fillna(2)
user_gender.head()

,user_id,gender
0,376517,1.0
1,234512,0.0
2,344532,0.0
3,186135,0.0
4,30230,0.0


In [10]:
user_gender_info = label_binarize(np.array(user_gender.gender), classes=[0,1,2])
user_gender_info = pd.DataFrame(data=user_gender_info, columns=['gender_female','gender_male','gender_unknown'])
user_gender['user_gender_female']=user_gender_info['gender_female']
user_gender['user_gender_male']=user_gender_info['gender_male']
user_gender['user_gender_unknown']=user_gender_info['gender_unknown']
user_gender = user_gender.drop(columns = ['gender'])

In [11]:
user_gender.head()

,user_id,user_gender_female,user_gender_male,user_gender_unknown
0,376517,0,1,0
1,234512,1,0,0
2,344532,1,0,0
3,186135,1,0,0
4,30230,1,0,0


In [12]:
user_age = user_info[['user_id','age_range']].copy()
user_age = user_age.fillna(9)
user_age = user_age.replace(8,7)
user_age = user_age.drop_duplicates()
user_age_info = label_binarize(np.array(user_age.age_range),classes=[1,2,3,4,5,6,7,9])
user_age_info = pd.DataFrame(data= user_age_info, columns = ['age_0_18','age_18_24','age_25_29','age_30_34','age_35_39','age_40_49','age_50_','age_unknown'])
user_age['user_age_0_18']=user_age_info['age_0_18']
user_age['user_age_18_24']=user_age_info['age_18_24']
user_age['user_age_25_29']=user_age_info['age_25_29']
user_age['user_age_30_34']=user_age_info['age_30_34']
user_age['user_age_35_39']=user_age_info['age_35_39']
user_age['user_age_40_49']=user_age_info['age_40_49']
user_age['user_age_50_']=user_age_info['age_50_']
user_age['user_age_unknown']=user_age_info['age_unknown']
user_age = user_age.drop(columns=['age_range'])
user_age.head()

,user_id,user_age_0_18,user_age_18_24,user_age_25_29,user_age_30_34,user_age_35_39,user_age_40_49,user_age_50_,user_age_unknown
0,376517,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,234512,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,344532,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,186135,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,30230,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Action count

In [13]:
user_action = user_log.groupby('user_id').size().to_frame().reset_index()
user_action.columns = ['user_id','user_total_action']
user_action.head()

,user_id,user_total_action
0,1,33
1,2,63
2,3,68
3,4,50
4,5,173


In [14]:
seller_action = user_log.groupby('seller_id').size().to_frame().reset_index()
seller_action.columns = ['seller_id','seller_total_action']
seller_action.head()

,seller_id,seller_total_action
0,1,339140
1,2,2371
2,3,2645
3,4,3106
4,5,8192


In [17]:
user_seller_action = user_log.groupby(['user_id','seller_id']).size().to_frame().reset_index()
user_seller_action.columns = ['user_id','seller_id','user_seller_total_action']
user_seller_action.head()

,user_id,seller_id,user_seller_total_action
0,1,471,1
1,1,739,1
2,1,925,4
3,1,1019,14
4,1,1156,1


In [18]:
user_activity = user_log[['user_id','action_type']].copy()
user_activity_info = label_binarize(np.array(user_activity.action_type), classes=[0,1,2,3])
user_activity_info = pd.DataFrame(data = user_activity_info, columns=['Click','add_to_cart','purchase','add_to_favorite'])
user_activity['user_Click']= user_activity_info.Click
user_activity['user_Add_to_cart']=user_activity_info.add_to_cart
user_activity['user_purchase']=user_activity_info.purchase
user_activity['user_Add_to_favorite']=user_activity_info.add_to_favorite
user_activity_count = user_activity.groupby('user_id').agg('sum').reset_index()
user_activity_count.columns = ['user_id','action_type','user_Click_Count','user_Add_to_cart_count','user_purchase_count','user_Add_to_favorite_count']
user_activity_count = user_activity_count.drop(columns=['action_type'])
user_activity_count.head()

,user_id,user_Click_Count,user_Add_to_cart_count,user_purchase_count,user_Add_to_favorite_count
0,1,27,0,6,0
1,2,47,0,14,2
2,3,63,0,4,1
3,4,49,0,1,0
4,5,150,0,13,10


In [19]:
seller_activity = user_log[['seller_id','action_type']].copy()
seller_activity_info = label_binarize(np.array(seller_activity.action_type), classes=[0,1,2,3])
seller_activity_info = pd.DataFrame(data = seller_activity_info, columns=['Click','add_to_cart','purchase','add_to_favorite'])
seller_activity['seller_Click']= seller_activity_info.Click
seller_activity['seller_Add_to_cart']=seller_activity_info.add_to_cart
seller_activity['seller_purchase']=seller_activity_info.purchase
seller_activity['seller_Add_to_favorite']=seller_activity_info.add_to_favorite
seller_activity_count = seller_activity.groupby('seller_id').agg('sum').reset_index()
seller_activity_count.columns = ['seller_id','action_type','seller_Click_Count','seller_Add_to_cart_count','seller_purchase_count','seller_Add_to_favorite_count']
seller_activity_count = seller_activity_count.drop(columns=['action_type'])
seller_activity_count.head()

,seller_id,seller_Click_Count,seller_Add_to_cart_count,seller_purchase_count,seller_Add_to_favorite_count
0,1,308236,444,17705,12755
1,2,2030,8,189,144
2,3,2399,4,67,175
3,4,2646,2,294,164
4,5,7483,9,144,556


In [20]:
user_seller_item = user_log[['user_id','seller_id','action_type']].copy()
user_seller_item_info = label_binarize(np.array(user_seller_item.action_type),classes=[0,1,2,3])
user_seller_item_info = pd.DataFrame(data= user_seller_item_info, columns=['Click','Add_to_cart','Purchase','Add_to_favorite'])
user_seller_item['user_seller_Click_count']= user_seller_item_info.Click
user_seller_item['user_seller_Add_to_cart_count']=user_seller_item_info.Add_to_cart
user_seller_item['user_seller_Purchase_count']=user_seller_item_info.Purchase
user_seller_item['user_seller_Add_to_favorite_count']=user_seller_item_info.Add_to_favorite
user_seller_item_act_count = user_seller_item.groupby(['user_id','seller_id']).agg('sum').reset_index()
user_seller_item_act_count = user_seller_item_act_count.drop(columns = ['action_type'])
user_seller_item_act_count.head()

,user_id,seller_id,user_seller_Click_count,user_seller_Add_to_cart_count,user_seller_Purchase_count,user_seller_Add_to_favorite_count
0,1,471,1,0,0,0
1,1,739,1,0,0,0
2,1,925,3,0,1,0
3,1,1019,10,0,4,0
4,1,1156,1,0,0,0


## Day count

In [21]:
user_active_days = user_log[['user_id','time_stamp']].drop_duplicates()
user_active_days_info = user_active_days.groupby('user_id').size().reset_index()
user_active_days_info.columns = ['user_id','user_active_days']
user_active_days_info.head()

,user_id,user_active_days
0,1,5
1,2,9
2,3,13
3,4,10
4,5,30


In [22]:
seller_active_days = user_log[['seller_id','time_stamp']].drop_duplicates()
seller_active_days_info = seller_active_days.groupby('seller_id').size().reset_index()
seller_active_days_info.columns = ['seller_id','seller_active_days']
seller_active_days_info.head()

,seller_id,seller_active_days
0,1,185
1,2,177
2,3,172
3,4,180
4,5,184


In [23]:
# user_active_days_2 = user_active_days
# user_active_days_2['Day']=user_active_days_2['time_stamp']%100
# user_active_days_2['Month']=((user_active_days_2['time_stamp']-user_active_days_2['Day'])/100).astype('int')
# user_active_days_2_info = label_binarize(np.array(user_active_days_2.Month),classes=[5,6,7,8,9,10,11])
# user_active_days_2_info = pd.DataFrame(data = user_active_days_2_info, columns=['May','June','July','Aug','Sept','Oct','Nov'])
# user_active_days_2['May'] = user_active_days_2_info.May
# user_active_days_2['June']= user_active_days_2_info.June
# user_active_days_2['July']=user_active_days_2_info.July
# user_active_days_2['Aug']=user_active_days_2_info.Aug
# user_active_days_2['Sept']= user_active_days_2_info.Sept
# user_active_days_2['Oct']= user_active_days_2_info.Oct
# user_active_days_2['Nov']=user_active_days_2_info.Nov
# user_active_Month_counts = user_active_days_2.groupby('user_id').agg('sum').reset_index()
# user_active_Month_counts = user_active_Month_counts.drop(columns = ['time_stamp','Day','Month'])
# user_active_Month_counts.columns = ['user_id', 'user_May_active_days','user_June_active_days','user_July_active_days','user_Aug_active_days','user_Sept_active_days','user_Oct_active_days','user_Nov_active_days']
# user_active_Month_counts.head()

In [24]:
user_seller_days = user_log[['user_id','seller_id','time_stamp']].copy()
user_seller_days.drop_duplicates(inplace=True)
user_seller_days = user_seller_days.groupby(['user_id','seller_id']).size().reset_index()
user_seller_days.columns = ['user_id','seller_id','user_seller_active_days']
user_seller_days.head()

,user_id,seller_id,user_seller_active_days
0,1,471,1
1,1,739,1
2,1,925,1
3,1,1019,1
4,1,1156,1


In [25]:
# user_seller_active_days = user_log[['user_id','seller_id','time_stamp']].copy().drop_duplicates()
# user_seller_active_days['Day']=user_seller_active_days['time_stamp']%100
# user_seller_active_days['Month']=((user_seller_active_days['time_stamp']-user_seller_active_days['Day'])/100).astype('int')
# us_active_days_info = label_binarize(np.array(user_seller_active_days.Month),classes=[5,6,7,8,9,10,11])
# us_active_days_info = pd.DataFrame(data = us_active_days_info, columns=['May','June','July','Aug','Sept','Oct','Nov'])
# user_seller_active_days['May'] = us_active_days_info.May
# user_seller_active_days['June']= us_active_days_info.June
# user_seller_active_days['July']=us_active_days_info.July
# user_seller_active_days['Aug']=us_active_days_info.Aug
# user_seller_active_days['Sept']= us_active_days_info.Sept
# user_seller_active_days['Oct']= us_active_days_info.Oct
# user_seller_active_days['Nov']=us_active_days_info.Nov
# user_seller_Month_counts = user_seller_active_days.groupby(['user_id','seller_id']).agg('sum').reset_index()
# user_seller_Month_counts.drop(columns=['time_stamp','Day','Month'],inplace=True)
# user_seller_Month_counts.columns = ['user_id','seller_id','user_seller_May_active_days','user_seller_June_active_days','user_seller_July_active_days','user_seller_aug_active_days','user_seller_sept_active_days','user_seller_oct_active_days','user_seller_nov_active_days']
# user_seller_Month_counts.head()

## Product diversity

In [26]:
seller_item = user_log[['seller_id','item_id']].copy()
seller_item.drop_duplicates(['item_id'], inplace=True)
seller_item_info = seller_item.groupby('seller_id').size().reset_index()
seller_item_info.columns = ['seller_id','seller_item_count']
seller_item_info.head()                                     

,seller_id,seller_item_count
0,1,2977
1,2,154
2,3,171
3,4,155
4,5,660


In [27]:
seller_brand = user_log[['seller_id','brand_id']].copy()
seller_brand.drop_duplicates(['brand_id'], inplace=True)
seller_brand_info = seller_brand.groupby('seller_id').size().reset_index()
seller_brand_info.columns = ['seller_id','seller_brand_count']
seller_brand_info.head()

,seller_id,seller_brand_count
0,1,2
1,3,1
2,4,1
3,5,1
4,6,1


In [28]:
seller_cat = user_log[['seller_id','cat_id']].copy()
seller_cat.drop_duplicates(['cat_id'], inplace=True)
seller_cat_info = seller_cat.groupby('seller_id').size().reset_index()
seller_cat_info.columns = ['seller_id','seller_category_count']
seller_cat_info.head()

,seller_id,seller_category_count
0,1,1
1,15,1
2,16,1
3,19,1
4,21,1


In [29]:
user_item = user_log[['user_id','item_id']].copy()
user_item.drop_duplicates(['item_id'],inplace=True)
user_item_info = user_item.groupby('user_id').size().reset_index()
user_item_info.columns = ['user_id','user_item_count']
user_item_info.head()

,user_id,user_item_count
0,5,2
1,6,3
2,9,1
3,10,1
4,14,2


In [30]:
user_cat = user_log[['user_id','cat_id']].copy()
user_cat.drop_duplicates(['cat_id'],inplace=True)
user_cat_info = user_cat.groupby('user_id').size().reset_index()
user_cat_info.columns = ['user_id','user_category_count']
user_cat_info.head()

,user_id,user_category_count
0,16,2
1,19,2
2,62,1
3,941,1
4,1920,1


In [31]:
user_brand = user_log[['user_id','brand_id']].copy()
user_brand.drop_duplicates(['brand_id'],inplace=True)
user_brand_info = user_cat.groupby('user_id').size().reset_index()
user_brand_info.columns = ['user_id','user_brand_count']
user_brand_info.head()

,user_id,user_brand_count
0,16,2
1,19,2
2,62,1
3,941,1
4,1920,1


In [32]:
brand_cat = user_log[['brand_id','cat_id']].copy()
brand_cat.drop_duplicates(['brand_id','cat_id'], inplace=True)
brand_cat_info = brand_cat.groupby('brand_id').size().reset_index()
brand_cat_info.columns = ['brand_id','brand_category_count']
brand_cat_info.head()

,brand_id,brand_category_count
0,1.0,17
1,2.0,1
2,3.0,1
3,4.0,1
4,5.0,11


In [33]:
cat_item = user_log[['cat_id','item_id']].copy()
cat_item.drop_duplicates(['cat_id','item_id'], inplace=True)
cat_item_info = cat_item.groupby('cat_id').size().reset_index()
cat_item_info.columns = ['cat_id','category_item_count']
cat_item_info.head()

,cat_id,category_item_count
0,1,34
1,2,2303
2,3,12
3,4,552
4,5,551


In [34]:
user_seller_item = user_log[['user_id','seller_id','item_id']].copy()
user_seller_item.drop_duplicates(inplace=True)
user_seller_item = user_seller_item.groupby(['user_id','seller_id']).size().reset_index()
user_seller_item.columns = ['user_id','seller_id','user_seller_item_count']
user_seller_item.head()

,user_id,seller_id,user_seller_item_count
0,1,471,1
1,1,739,1
2,1,925,1
3,1,1019,1
4,1,1156,1


In [35]:
user_seller_brand = user_log[['user_id','seller_id','brand_id']].copy()
user_seller_brand.drop_duplicates(inplace=True)
user_seller_brand = user_seller_brand.groupby(['user_id','seller_id']).size().reset_index()
user_seller_brand.columns = ['user_id','seller_id','user_seller_brand_count']
user_seller_brand.head()

,user_id,seller_id,user_seller_brand_count
0,1,471,1
1,1,739,1
2,1,925,1
3,1,1019,1
4,1,1156,1


In [36]:
user_seller_cat = user_log[['user_id','seller_id','cat_id']].copy()
user_seller_cat.drop_duplicates(inplace=True)
user_seller_cat = user_seller_cat.groupby(['user_id','seller_id']).size().reset_index()
user_seller_cat.columns = ['user_id','seller_id','user_seller_category_count']
user_seller_cat.head()

,user_id,seller_id,user_seller_category_count
0,1,471,1
1,1,739,1
2,1,925,1
3,1,1019,1
4,1,1156,1


## User-merchant similarity

In [37]:
# merchant’s market share on the brand = NMB/NB
seller_mkt_share_brand = user_log[['seller_id','brand_id']].copy()
seller_mkt_share_brand = seller_mkt_share_brand.groupby(['seller_id','brand_id']).size().reset_index()
seller_mkt_share_brand.columns = ['seller_id','brand_id','NMB']
seller_mkt_share_brand['seller_mkt_share_brand'] = seller_mkt_share_brand['NMB']/seller_mkt_share_brand.groupby('brand_id')['NMB'].transform('sum')
seller_mkt_share_brand = seller_mkt_share_brand.drop(columns=['NMB'])
seller_mkt_share_brand.head()

,seller_id,brand_id,seller_mkt_share_brand
0,1,1104.0,1.000000
1,1,1662.0,0.996967
2,2,2921.0,0.283946
3,3,970.0,1.000000
4,4,1944.0,1.000000


In [38]:
# user-merchant similarity on brand
user_mkt_share_brand = user_log.groupby(['user_id','brand_id']).size().reset_index()
user_mkt_share_brand.columns = ['user_id','brand_id','NUB']
user_seller_brand_similarity = user_log[['user_id','seller_id','brand_id']].copy()
user_seller_brand_similarity.drop_duplicates(inplace=True)
user_seller_brand_similarity = pd.merge(user_seller_brand_similarity,seller_mkt_share_brand,on=['seller_id','brand_id'],how='left')
user_seller_brand_similarity = pd.merge(user_seller_brand_similarity,user_mkt_share_brand,on=['user_id','brand_id'],how='left')
user_seller_brand_similarity.fillna(0)
user_seller_brand_similarity['brand_similarity_score'] = user_seller_brand_similarity['seller_mkt_share_brand']*user_seller_brand_similarity['NUB']
user_seller_brand_similarity = user_seller_brand_similarity.groupby(['user_id','seller_id'])['brand_similarity_score'].agg('sum').reset_index()
user_seller_brand_similarity.head()

,user_id,seller_id,brand_similarity_score
0,1,471,1.000000
1,1,739,1.000000
2,1,925,2.402976
3,1,1019,14.000000
4,1,1156,1.000000


In [39]:
# user-merchant similarity on category
seller_mkt_share_cat = user_log[['seller_id','cat_id']].copy()
seller_mkt_share_cat = seller_mkt_share_cat.groupby(['seller_id','cat_id']).size().reset_index()
seller_mkt_share_cat.columns = ['seller_id','cat_id','NMC']
seller_mkt_share_cat['seller_mkt_share_cat'] = seller_mkt_share_cat['NMC']/seller_mkt_share_cat.groupby('cat_id')['NMC'].transform('sum')
seller_mkt_share_cat = seller_mkt_share_cat.drop(columns=['NMC'])

user_mkt_share_cat = user_log.groupby(['user_id','cat_id']).size().reset_index()
user_mkt_share_cat.columns = ['user_id','cat_id','NUC']

user_seller_cat_similarity = user_log[['user_id','seller_id','cat_id']].copy()
user_seller_cat_similarity.drop_duplicates(inplace=True)
user_seller_cat_similarity = pd.merge(user_seller_cat_similarity,seller_mkt_share_cat,on=['seller_id','cat_id'],how='left')
user_seller_cat_similarity = pd.merge(user_seller_cat_similarity,user_mkt_share_cat,on=['user_id','cat_id'],how='left')
user_seller_cat_similarity.fillna(0)
user_seller_cat_similarity['cat_similarity_score'] = user_seller_cat_similarity['seller_mkt_share_cat']*user_seller_cat_similarity['NUC']
user_seller_cat_similarity = user_seller_cat_similarity.groupby(['user_id','seller_id'])['cat_similarity_score'].agg('sum').reset_index()
user_seller_cat_similarity.head()

,user_id,seller_id,cat_similarity_score
0,1,471,0.004445
1,1,739,0.086667
2,1,925,0.069735
3,1,1019,0.236661
4,1,1156,0.333996


In [40]:
# # merchant’s user share on the brand = UMB/UB
# seller_user_share_brand = user_log[['seller_id','brand_id','user_id']].copy()
# seller_user_share_brand.drop_duplicates(inplace=True)
# seller_user_share_brand = seller_user_share_brand.groupby(['seller_id','brand_id']).size().reset_index()
# seller_user_share_brand.columns = ['seller_id','brand_id','UMB']
# seller_user_share_brand['seller_user_share_brand'] = seller_user_share_brand['UMB']/seller_user_share_brand.groupby('brand_id')['UMB'].transform('sum')
# seller_user_share_brand = seller_user_share_brand.drop(columns=['UMB'])
# seller_user_share_brand.head()

In [41]:
# # brand’s market share within the merchant = NMB/NM
# brand_mkt_share_seller = user_log[['seller_id','brand_id']].copy()
# brand_mkt_share_seller = brand_mkt_share_seller.groupby(['seller_id','brand_id']).size().reset_index()
# brand_mkt_share_seller.columns = ['seller_id','brand_id','NMB']
# brand_mkt_share_seller['brand_mkt_share_seller'] = brand_mkt_share_seller['NMB']/brand_mkt_share_seller.groupby('seller_id')['NMB'].transform('sum')
# brand_mkt_share_seller = brand_mkt_share_seller.drop(columns=['NMB'])
# brand_mkt_share_seller.head()

In [42]:
# # brand’s user share within the merchant = UMB/UM
# brand_user_share_seller = user_log[['seller_id','brand_id','user_id']].copy()
# brand_user_share_seller.drop_duplicates(inplace=True)
# brand_user_share_seller = brand_user_share_seller.groupby(['seller_id','brand_id']).size().reset_index()
# brand_user_share_seller.columns = ['seller_id','brand_id','UMB']
# brand_user_share_seller['brand_user_share_seller'] = brand_user_share_seller['UMB']/brand_user_share_seller.groupby('brand_id')['UMB'].transform('sum')
# brand_user_share_seller = brand_user_share_seller.drop(columns=['UMB'])
# brand_user_share_seller.head()

## Recent activity

In [43]:
user_time_period = user_log[['user_id','seller_id','time_stamp','action_type']].copy()
user_time_period['Day']=user_time_period['time_stamp']%100
user_time_period['Month']=((user_time_period['time_stamp']-user_time_period['Day'])/100).astype('int')
user_time_period['user_seller_Click_count']= user_seller_item_info.Click
user_time_period['user_seller_Add_to_cart_count']=user_seller_item_info.Add_to_cart
user_time_period['user_seller_Purchase_count']=user_seller_item_info.Purchase
user_time_period['user_seller_Add_to_favorite_count']=user_seller_item_info.Add_to_favorite
user_time_period.head()

,user_id,seller_id,time_stamp,action_type,Day,Month,user_seller_Click_count,user_seller_Add_to_cart_count,user_seller_Purchase_count,user_seller_Add_to_favorite_count
0,328862,2882,829,0,29,8,1,0,0,0
1,328862,2882,829,0,29,8,1,0,0,0
2,328862,2882,829,0,29,8,1,0,0,0
3,328862,2882,829,0,29,8,1,0,0,0
4,328862,1253,829,0,29,8,1,0,0,0


In [44]:
user_time_period['Dummy_Year'] = 2020
user_time_period['Date'] = pd.to_datetime(user_time_period['Dummy_Year'].astype(str)+user_time_period['Month'].astype(str)+user_time_period['Day'].astype(str), format = '%Y%m%d')
double_11 = user_time_period[user_time_period['Date'] == '2020-11-11']
one_week_double_11 = user_time_period[(user_time_period['Date']>'2020-11-04') & (user_time_period['Date']<'2020-11-11')]
one_month_double_11 = user_time_period[(user_time_period['Date']>'2020-10-04') & (user_time_period['Date']<='2020-11-04')]
two_month_double_11 = user_time_period[(user_time_period['Date']>'2020-09-04') & (user_time_period['Date']<='2020-10-04')]
three_month_double_11 = user_time_period[(user_time_period['Date']>'2020-08-04') & (user_time_period['Date']<='2020-09-04')]
more_than_three_month_double_11 = user_time_period[(user_time_period['Date']<='2020-08-04')]
user_time_period.head()

,user_id,seller_id,time_stamp,action_type,Day,Month,user_seller_Click_count,user_seller_Add_to_cart_count,user_seller_Purchase_count,user_seller_Add_to_favorite_count,Dummy_Year,Date
0,328862,2882,829,0,29,8,1,0,0,0,2020,2020-08-29
1,328862,2882,829,0,29,8,1,0,0,0,2020,2020-08-29
2,328862,2882,829,0,29,8,1,0,0,0,2020,2020-08-29
3,328862,2882,829,0,29,8,1,0,0,0,2020,2020-08-29
4,328862,1253,829,0,29,8,1,0,0,0,2020,2020-08-29


In [45]:
# double 11
double_11_info = label_binarize(np.array(double_11.action_type),classes=[0,1,2,3])
double_11_info = pd.DataFrame(data= double_11_info, columns=['Click','Add_to_cart','Purchase','Add_to_favorite'])
double_11['user_seller_Click_count']= double_11_info.Click
double_11['user_seller_Add_to_cart_count']=double_11_info.Add_to_cart
double_11['user_seller_Purchase_count']=double_11_info.Purchase
double_11['user_seller_Add_to_favorite_count']=double_11_info.Add_to_favorite
double_11_act_count = double_11.groupby(['user_id','seller_id']).agg('sum').reset_index()
double_11_act_count = double_11_act_count.drop(columns = ['action_type'])
double_11_act_count.head()

<ipython-input-45-a63bf1000c88>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  double_11['user_seller_Click_count']= double_11_info.Click
<ipython-input-45-a63bf1000c88>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  double_11['user_seller_Add_to_cart_count']=double_11_info.Add_to_cart
<ipython-input-45-a63bf1000c88>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,user_id,seller_id,time_stamp,Day,Month,user_seller_Click_count,user_seller_Add_to_cart_count,user_seller_Purchase_count,user_seller_Add_to_favorite_count,Dummy_Year
0,1,471,1111,11,11,0.0,0.0,0.0,0.0,2020
1,1,1019,15554,154,154,0.0,0.0,0.0,0.0,28280
2,1,1156,1111,11,11,0.0,0.0,0.0,0.0,2020
3,1,4335,1111,11,11,0.0,0.0,0.0,0.0,2020
4,2,1179,1111,11,11,0.0,0.0,0.0,0.0,2020


In [46]:
# latest one-week
one_week_info = label_binarize(np.array(one_week_double_11.action_type),classes=[0,1,2,3])
one_week_info = pd.DataFrame(data= one_week_info, columns=['Click','Add_to_cart','Purchase','Add_to_favorite'])
one_week_double_11['user_seller_Click_count']= one_week_info.Click
one_week_double_11['user_seller_Add_to_cart_count']=one_week_info.Add_to_cart
one_week_double_11['user_seller_Purchase_count']=one_week_info.Purchase
one_week_double_11['user_seller_Add_to_favorite_count']=one_week_info.Add_to_favorite
one_week_act_count = one_week_double_11.groupby(['user_id','seller_id']).agg('sum').reset_index()
one_week_act_count = one_week_act_count.drop(columns = ['action_type'])
one_week_act_count.head()

<ipython-input-46-b52b83e900c7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_week_double_11['user_seller_Click_count']= one_week_info.Click
<ipython-input-46-b52b83e900c7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_week_double_11['user_seller_Add_to_cart_count']=one_week_info.Add_to_cart
<ipython-input-46-b52b83e900c7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,user_id,seller_id,time_stamp,Day,Month,user_seller_Click_count,user_seller_Add_to_cart_count,user_seller_Purchase_count,user_seller_Add_to_favorite_count,Dummy_Year
0,2,2412,1108,8,11,0.0,0.0,0.0,0.0,2020
1,3,493,2218,18,22,0.0,0.0,0.0,0.0,4040
2,3,806,16632,132,165,0.0,0.0,0.0,0.0,30300
3,3,1897,1109,9,11,0.0,0.0,0.0,0.0,2020
4,3,2313,5545,45,55,0.0,0.0,0.0,0.0,10100


In [47]:
# latest one-month
one_month_info = label_binarize(np.array(one_month_double_11.action_type),classes=[0,1,2,3])
one_month_info = pd.DataFrame(data= one_month_info, columns=['Click','Add_to_cart','Purchase','Add_to_favorite'])
one_month_double_11['user_seller_Click_count']= one_month_info.Click
one_month_double_11['user_seller_Add_to_cart_count']=one_month_info.Add_to_cart
one_month_double_11['user_seller_Purchase_count']=one_month_info.Purchase
one_month_double_11['user_seller_Add_to_favorite_count']=one_month_info.Add_to_favorite
one_month_act_count = one_month_double_11.groupby(['user_id','seller_id']).agg('sum').reset_index()
one_month_act_count = one_month_double_11.drop(columns = ['action_type'])
one_month_act_count.head()

<ipython-input-47-c2610ee95207>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_month_double_11['user_seller_Click_count']= one_month_info.Click
<ipython-input-47-c2610ee95207>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_month_double_11['user_seller_Add_to_cart_count']=one_month_info.Add_to_cart
<ipython-input-47-c2610ee95207>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,user_id,seller_id,time_stamp,Day,Month,user_seller_Click_count,user_seller_Add_to_cart_count,user_seller_Purchase_count,user_seller_Add_to_favorite_count,Dummy_Year,Date
56,328862,4043,1013,13,10,1.0,0.0,0.0,0.0,2020,2020-10-13
62,328862,4043,1013,13,10,1.0,0.0,0.0,0.0,2020,2020-10-13
126,328862,1138,1102,2,11,1.0,0.0,0.0,0.0,2020,2020-11-02
127,328862,1138,1102,2,11,0.0,0.0,0.0,1.0,2020,2020-11-02
128,328862,3828,1102,2,11,1.0,0.0,0.0,0.0,2020,2020-11-02


In [48]:
# #two month before double 11
# two_month_info = label_binarize(np.array(two_month_double_11.action_type),classes=[0,1,2,3])
# two_month_info = pd.DataFrame(data= two_month_info, columns=['Click','Add_to_cart','Purchase','Add_to_favorite'])
# two_month_double_11['user_seller_Click_count']= two_month_info.Click
# two_month_double_11['user_seller_Add_to_cart_count']=two_month_info.Add_to_cart
# two_month_double_11['user_seller_Purchase_count']=two_month_info.Purchase
# two_month_double_11['user_seller_Add_to_favorite_count']=two_month_info.Add_to_favorite
# two_month_act_count = two_month_double_11.groupby(['user_id','seller_id']).agg('sum').reset_index()
# two_month_act_count = two_month_double_11.drop(columns = ['action_type'])
# two_month_act_count.head()

In [49]:
# #Three month before double 11
# three_month_info = label_binarize(np.array(three_month_double_11.action_type),classes=[0,1,2,3])
# three_month_info = pd.DataFrame(data= three_month_info, columns=['Click','Add_to_cart','Purchase','Add_to_favorite'])
# three_month_double_11['user_seller_Click_count']= three_month_info.Click
# three_month_double_11['user_seller_Add_to_cart_count']=three_month_info.Add_to_cart
# three_month_double_11['user_seller_Purchase_count']=three_month_info.Purchase
# three_month_double_11['user_seller_Add_to_favorite_count']=three_month_info.Add_to_favorite
# three_month_act_count = three_month_double_11.groupby(['user_id','seller_id']).agg('sum').reset_index()
# three_month_act_count = three_month_double_11.drop(columns = ['action_type'])
# three_month_act_count.head()

In [50]:
# #more than three month before double 11
# more_than_three_info = label_binarize(np.array(more_than_three_month_double_11.action_type),classes=[0,1,2,3])
# more_than_three_info = pd.DataFrame(data= more_than_three_info, columns=['Click','Add_to_cart','Purchase','Add_to_favorite'])
# more_than_three_month_double_11['user_seller_Click_count']= more_than_three_info.Click
# more_than_three_month_double_11['user_seller_Add_to_cart_count']=more_than_three_info.Add_to_cart
# more_than_three_month_double_11['user_seller_Purchase_count']=more_than_three_info.Purchase
# more_than_three_month_double_11['user_seller_Add_to_favorite_count']=more_than_three_info.Add_to_favorite
# more_than_three_act_count = more_than_three_month_double_11.groupby(['user_id','seller_id']).agg('sum').reset_index()
# more_than_three_act_count = more_than_three_month_double_11.drop(columns = ['action_type'])
# more_than_three_act_count.head()

In [51]:
# train_df = pd.read_csv('C:/Stats Consulting/train.csv')
train_df = pd.read_csv('/Users/joe/study/2401-statistical consulting/data/train_data.csv')
train_df.head()

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [52]:
train_df.shape

(260864, 3)

In [53]:
# test_df = pd.read_csv('C:/Stats Consulting/Test.csv')
test_df = pd.read_csv('/Users/joe/study/2401-statistical consulting/data/test_data.csv')
test_df.head()

,user_id,merchant_id,prob
0,163968,4605,NaN
1,360576,1581,NaN
2,98688,1964,NaN
3,98688,3645,NaN
4,295296,3361,NaN


In [54]:
test_df.shape

(261477, 3)

In [55]:
# user-level feature

# demographic
user_level_feature = pd.merge(user_gender, user_age, on=['user_id'],how='left')

# counts
# day counts
user_level_feature = pd.merge(user_level_feature, user_active_days_info, on=['user_id'],how='left')
# action counts
user_level_feature = pd.merge(user_level_feature, user_action, on=['user_id'],how='left')
user_level_feature = pd.merge(user_level_feature, user_activity_count, on=['user_id'],how='left')

# product diversity
user_level_feature = pd.merge(user_level_feature, user_item_info, on=['user_id'],how='left')
user_level_feature = pd.merge(user_level_feature, user_cat_info, on=['user_id'],how='left')
user_level_feature = pd.merge(user_level_feature, user_brand_info, on=['user_id'],how='left')

user_level_feature.head()

,user_id,user_gender_female,user_gender_male,user_gender_unknown,user_age_0_18,user_age_18_24,user_age_25_29,user_age_30_34,user_age_35_39,user_age_40_49,...,user_age_unknown,user_active_days,user_total_action,user_Click_Count,user_Add_to_cart_count,user_purchase_count,user_Add_to_favorite_count,user_item_count,user_category_count,user_brand_count
0,376517,0,1,0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,10,79,73,0,5,1,NaN,NaN,NaN
1,234512,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,11,56,47,0,7,2,29.0,11.0,11.0
2,344532,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,14,91,64,0,8,19,1.0,NaN,NaN
3,186135,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,5,200,193,0,2,5,1.0,NaN,NaN
4,30230,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,13,32,29,0,3,0,NaN,NaN,NaN


In [56]:
user_level_feature.shape

(424170, 21)

In [57]:
# seller-level feature

# product diversity
seller_level_feature = pd.merge(seller_item_info, seller_cat_info, on=['seller_id'],how='left')
seller_level_feature = pd.merge(seller_level_feature, seller_brand_info, on=['seller_id'],how='left')

# counts
# day counts
seller_level_feature = pd.merge(seller_level_feature, seller_active_days_info, on=['seller_id'],how='left')
# action counts
seller_level_feature = pd.merge(seller_level_feature, seller_action, on=['seller_id'],how='left')
seller_level_feature = pd.merge(seller_level_feature, seller_activity_count, on=['seller_id'],how='left')

seller_level_feature.head()

,seller_id,seller_item_count,seller_category_count,seller_brand_count,seller_active_days,seller_total_action,seller_Click_Count,seller_Add_to_cart_count,seller_purchase_count,seller_Add_to_favorite_count
0,1,2977,1.0,2.0,185,339140,308236,444,17705,12755
1,2,154,NaN,NaN,177,2371,2030,8,189,144
2,3,171,NaN,1.0,172,2645,2399,4,67,175
3,4,155,NaN,1.0,180,3106,2646,2,294,164
4,5,660,NaN,1.0,184,8192,7483,9,144,556


In [58]:
seller_level_feature.shape

(4995, 10)

In [59]:
# user-seller interaction level feature

# product diversity
user_seller_level_feature = pd.merge(user_seller_item, user_seller_cat, on=['user_id','seller_id'],how='left')
user_seller_level_feature = pd.merge(user_seller_level_feature, user_seller_brand, on=['user_id','seller_id'],how='left')

# counts
# action counts
user_seller_level_feature = pd.merge(user_seller_level_feature, user_seller_item_act_count, on=['user_id','seller_id'],how='left')
user_seller_level_feature = pd.merge(user_seller_level_feature, user_seller_action, on=['user_id','seller_id'],how='left')
# day counts
user_seller_level_feature = pd.merge(user_seller_level_feature, user_seller_days,on=['user_id','seller_id'],how='left') 

# similarity
user_seller_level_feature = pd.merge(user_seller_level_feature,user_seller_brand_similarity,on=['user_id','seller_id'],how='left') 
user_seller_level_feature = pd.merge(user_seller_level_feature,user_seller_cat_similarity,on=['user_id','seller_id'],how='left')

# recent activity
user_seller_level_feature = pd.merge(user_seller_level_feature,double_11_act_count,on=['user_id','seller_id'],how='left') 
user_seller_level_feature = pd.merge(user_seller_level_feature,one_week_act_count,on=['user_id','seller_id'],how='left') 
user_seller_level_feature = pd.merge(user_seller_level_feature,one_month_act_count,on=['user_id','seller_id'],how='left') 

user_seller_level_feature.head()

,user_id,seller_id,user_seller_item_count,user_seller_category_count,user_seller_brand_count,user_seller_Click_count_x,user_seller_Add_to_cart_count_x,user_seller_Purchase_count_x,user_seller_Add_to_favorite_count_x,user_seller_total_action,...,Dummy_Year_y,time_stamp,Day,Month,user_seller_Click_count_y,user_seller_Add_to_cart_count_y,user_seller_Purchase_count_y,user_seller_Add_to_favorite_count_y,Dummy_Year,Date
0,1,471,1,1,1,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,1,739,1,1,1,1,0,0,0,1,...,NaN,1018.0,18.0,10.0,NaN,NaN,NaN,NaN,2020.0,2020-10-18
2,1,925,1,1,1,3,0,1,0,4,...,NaN,1011.0,11.0,10.0,NaN,NaN,NaN,NaN,2020.0,2020-10-11
3,1,925,1,1,1,3,0,1,0,4,...,NaN,1011.0,11.0,10.0,NaN,NaN,NaN,NaN,2020.0,2020-10-11
4,1,925,1,1,1,3,0,1,0,4,...,NaN,1011.0,11.0,10.0,NaN,NaN,NaN,NaN,2020.0,2020-10-11


In [60]:
user_seller_level_feature.shape

(21190537, 38)

In [67]:
#combine train data and new added feature
#add user-level feature
train_complete = pd.merge(train_df, user_level_feature, on=['user_id'],how='left')
#add seller-level feature
seller_level_feature = seller_level_feature.rename(columns = {'seller_id':'merchant_id'})
train_complete = pd.merge(train_complete, seller_level_feature, on=['merchant_id'],how='left')
#add user-seller level feature
user_seller_level_feature = user_seller_level_feature.rename(columns = {'seller_id':'merchant_id'})
train_complete = pd.merge(train_complete, user_seller_level_feature, on=['user_id','merchant_id'],how='left')
train_complete.head()

AttributeError: module 'pandas' has no attribute 'write_csv'

In [68]:
train_complete.to_csv (r'train_complete.csv', index = False, header=True)

In [62]:
train_complete.shape

(521838, 68)

In [63]:
#combine test data and new added feature
#add user-level feature
test_complete = pd.merge(test_df, user_level_feature, on=['user_id'],how='left')
#add seller-level feature
test_complete = pd.merge(test_complete, seller_level_feature,on=['merchant_id'],how='left')
#add user-seller-level feature
test_complete = pd.merge(test_complete, user_seller_level_feature, on=['user_id','merchant_id'],how='left')
test_complete.head()


,user_id,merchant_id,prob,user_gender_female,user_gender_male,user_gender_unknown,user_age_0_18,user_age_18_24,user_age_25_29,user_age_30_34,...,Dummy_Year_y,time_stamp,Day,Month,user_seller_Click_count_y,user_seller_Add_to_cart_count_y,user_seller_Purchase_count_y,user_seller_Add_to_favorite_count_y,Dummy_Year,Date
0,163968,4605,NaN,1,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,360576,1581,NaN,0,0,1,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,98688,1964,NaN,1,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,98688,3645,NaN,1,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,295296,3361,NaN,0,1,0,0.0,1.0,0.0,0.0,...,52520.0,1101.0,1.0,11.0,NaN,NaN,NaN,NaN,2020.0,2020-11-01


In [64]:
test_complete.shape

(531247, 68)

In [69]:
test_complete.to_csv (r'test_complete.csv', index = False, header=True)